# Homework 2. Food safety

## Data reading & libraries importing

In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display
import matplotlib.pyplot as plt

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Set style for seaborn plots
sns.set_style('dark')
sns.color_palette("viridis", as_cmap=True);

In [5]:
ins = pd.read_csv('data\\ins.csv')       # Use double backslashes or a raw string
ins2vio = pd.read_csv('data\\ins2vio.csv')
legend = pd.read_csv('data/legend.csv')  # Use forward slashes or os.path.join
vio = pd.read_csv('data/vio.csv')
bus = pd.read_csv('data/bus.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data\\ins.csv'

In [ ]:
display(bus.head(1))
display(ins.head(1))
bus.describe()

,business id column,name,address,city,state,postal_code,latitude,longitude,phone_number
0,1000,HEUNG YUEN RESTAURANT,3279 22nd St,San Francisco,CA,94110,37.755282,-122.420493,-9999


,iid,date,score,type
0,100010_20190329,03/29/2019 12:00:00 AM,-1,New Construction


,business id column,latitude,longitude,phone_number
count,6253.000000,6253.000000,6253.000000,6.253000e+03
mean,60448.948984,-5575.337966,-5645.817699,4.701819e+09
std,36480.132445,4983.390142,4903.993683,6.667508e+09
min,19.000000,-9999.000000,-9999.000000,-9.999000e+03
25%,18399.000000,-9999.000000,-9999.000000,-9.999000e+03
50%,75685.000000,-9999.000000,-9999.000000,-9.999000e+03
75%,90886.000000,37.776494,-122.421553,1.415533e+10
max,102705.000000,37.824494,0.000000,1.415988e+10


# PART I.

After this homework, you should be comfortable with:
* Reading CSV files, 
* Reading `Pandas` documentation and using `Pandas`,
* Working with data at different levels of granularity,
* Identifying the type of data collected, missing values, anomalies, etc., and doing some basic analysis

---
## 1. Data loading

Importing zip files
1. import zipfile package
2. file_path
3. folder to extract the files to (extract)
4. with zipfile.ZipFile(file_path, 'r') as my_zip:
    my_zip.extractall(extract)

The block below X work cos I alr move the data files -> skip

In [ ]:
'''
import zipfile
zip_file_path = 'D:/Github/Python/data100/4homework/hw02A/data.zip'
extracted_dir = 'D:/Github/Python/data100/4homework/hw02A'

with zipfile.ZipFile(zip_file_path, 'r') as my_zip:
    my_zip.extractall(extracted_dir);
'''

"\nimport zipfile\nzip_file_path = 'D:/Github/Python/data100/4homework/hw02A/data.zip'\nextracted_dir = 'D:/Github/Python/data100/4homework/hw02A'\n\nwith zipfile.ZipFile(zip_file_path, 'r') as my_zip:\n    my_zip.extractall(extracted_dir);\n"

---
## [Advanced - zipfile and os] 

<details>
<summary>Click to expand/collapse</summary>

**Looking Inside and Extracting the Zip Files**

The following code blocks are for setup. Simply run the cells; **do not modify them**. Question 1a is where you will start to write code.

Here, we assign `my_zip` to a `zipfile.Zipfile` object representing `data.zip`, and assign `list_names` to a list of all the names of the contents in `data.zip`.

In [ ]:
'''
import zipfile
my_zip = zipfile.ZipFile(dest_path, 'r')
list_names = my_zip.namelist()
list_names
'''

"\nimport zipfile\nmy_zip = zipfile.ZipFile(dest_path, 'r')\nlist_names = my_zip.namelist()\nlist_names\n"

You may notice that we did not write `zipfile.ZipFile('data.zip', ...)`. Instead, we used `zipfile.ZipFile(dest_path, ...)`. In general, we **strongly suggest having your filenames hard coded as string literals only once** in a notebook. It is very dangerous to hardcode things twice because if you change one but forget to change the other, you can end up with bugs that are very hard to find.

Now, we display the files' names and their sizes.

In [ ]:
'''
my_zip = zipfile.ZipFile(dest_path, 'r')
for info in my_zip.infolist():
    print('{}\t{}'.format(info.filename, info.file_size))
'''

"\nmy_zip = zipfile.ZipFile(dest_path, 'r')\nfor info in my_zip.infolist():\n    print('{}\t{}'.format(info.filename, info.file_size))\n"

Often when working with zipped data, we'll never unzip the actual zipfile. This saves space on our local computer. However, for this homework the files are small, so we're just going to unzip everything. This has the added benefit that you can look inside the CSV files using a text editor, which might be handy for understanding the structure of the files. The cell below will unzip the CSV files into a sub-directory called `data`.

In [ ]:
'''
data_dir = Path('.')
my_zip.extractall(data_dir)
!ls {data_dir / Path("data")}
'''

'\ndata_dir = Path(\'.\')\nmy_zip.extractall(data_dir)\n!ls {data_dir / Path("data")}\n'

The cell above created a folder called `data`, and in it there should be five CSV files. Let's open up `legend.csv` to see its contents. To do this, click on the file icon on the top left to show the folders and files within the hw02A folder, then click on `legend.csv`. The file will open up in another tab. You should see something that looks like:

    "Minimum_Score","Maximum_Score","Description"
    0,70,"Poor"
    71,85,"Needs Improvement"
    86,90,"Adequate"
    91,100,"Good"

The `legend.csv` file does indeed look like a well-formed CSV file. Let's check the other three files. Rather than opening up each file manually, let's use Python to print out the first 5 lines of each. We defined a helper function for you that will allow you to retrieve the first N lines of a file as a list. For example, `head('data/legend.csv', 5)` will return the first 5 lines of "data/legend.csv". Run the cell below to print out the first 5 lines of all six files that we just extracted from the zipfile.

In [ ]:
'''
import os

def head(filename, lines=5):
    """
    Returns the first few lines of a file.
    
    filename: the name of the file to open
    lines: the number of lines to include
    
    return: A list of the first few lines from the file.
    """
    from itertools import islice
    with open(filename, "r") as f:
        return list(islice(f, lines))

data_dir = "./"
for f in list_names:
    if not os.path.isdir(f):
        print(head(data_dir + f, 5), "\n")
'''

'\nimport os\n\ndef head(filename, lines=5):\n    """\n    Returns the first few lines of a file.\n    \n    filename: the name of the file to open\n    lines: the number of lines to include\n    \n    return: A list of the first few lines from the file.\n    """\n    from itertools import islice\n    with open(filename, "r") as f:\n        return list(islice(f, lines))\n\ndata_dir = "./"\nfor f in list_names:\n    if not os.path.isdir(f):\n        print(head(data_dir + f, 5), "\n")\n'

</details>

First, we check the basic structure of the data frames you created:

In [ ]:
assert all(bus.columns == ['business id column', 'name', 'address', 'city', 'state', 'postal_code',
                           'latitude', 'longitude', 'phone_number'])
assert 6250 <= len(bus) <= 6260

assert all(ins.columns == ['iid', 'date', 'score', 'type'])
assert 26660 <= len(ins) <= 26670

assert all(vio.columns == ['description', 'risk_category', 'vid'])
assert 60 <= len(vio) <= 65

assert all(ins2vio.columns == ['iid', 'vid'])
assert 40210 <= len(ins2vio) <= 40220

Next we'll check that the statistics match what we expect. The following are hard-coded statistical summaries of the correct data.

In [ ]:
bus_summary = pd.DataFrame(**{'columns': ['business id column', 'latitude', 'longitude'],
 'data': {'business id column': {'50%': 75685.0, 'max': 102705.0, 'min': 19.0},
  'latitude': {'50%': -9999.0, 'max': 37.824494, 'min': -9999.0},
  'longitude': {'50%': -9999.0,
   'max': 0.0,
   'min': -9999.0}},
 'index': ['min', '50%', 'max']})

ins_summary = pd.DataFrame(**{'columns': ['score'],
 'data': {'score': {'50%': 76.0, 'max': 100.0, 'min': -1.0}},
 'index': ['min', '50%', 'max']})

vio_summary = pd.DataFrame(**{'columns': ['vid'],
 'data': {'vid': {'50%': 103135.0, 'max': 103177.0, 'min': 103102.0}},
 'index': ['min', '50%', 'max']})

from IPython.display import display

print('What we expect from your Businesses Dataframe:')
display(bus_summary)
print('What we expect from your Inspections Dataframe:')
display(ins_summary)
print('What we expect from your Violations Dataframe:')
display(vio_summary)

What we expect from your Businesses Dataframe:


,business id column,latitude,longitude
min,19.0,-9999.000000,-9999.0
50%,75685.0,-9999.000000,-9999.0
max,102705.0,37.824494,0.0


What we expect from your Inspections Dataframe:


,score
min,-1.0
50%,76.0
max,100.0


What we expect from your Violations Dataframe:


,vid
min,103102.0
50%,103135.0
max,103177.0


The code below defines a testing function that we'll use to verify that your data has the same statistics as what we expect. Run these cells to define the function. The `df_allclose` function has this name because we are verifying that all of the statistics for your Dataframe are close to the expected values. Why not `df_allequal`? It's a bad idea in almost all cases to compare two floating point values like 37.780435, as rounding errors can cause spurious failures. Run the following cells to load some basic utilities (you do not need to change these at all):

In [ ]:
def df_allclose(actual, desired, columns=None, rtol=5e-2):
    """Compare selected columns of two Dataframes on a few summary statistics.
    
    Compute the min, median and max of the two Dataframes on the given columns, and compare
    that they match numerically to the given relative tolerance.
    
    If they don't match, an AssertionError is raised (by `numpy.testing`).
    """    
    # Summary statistics to compare on
    stats = ['min', '50%', 'max']
    
    # For the desired values, we can provide a full DF with the same structure as
    # the actual data, or pre-computed summary statistics.
    # We assume a pre-computed summary was provided if column is None. In that case, 
    # `desired` *must* have the same structure as the actual's summary
    if columns is None:
        des = desired
        columns = desired.columns
    else:
        des = desired[columns].describe().loc[stats]

    # Extract summary stats from actual DF
    act = actual[columns].describe().loc[stats]

    return np.allclose(act, des, rtol)

We will now explore each file in turn, including determining its granularity and exploring many of the variables individually. Let's begin with the businesses file, which has been read into the `bus` Dataframe.

---
## Question 1. Examining the Business Data File

### Question 1a

In [ ]:
bus.head(1)

,business id column,name,address,city,state,postal_code,latitude,longitude,phone_number
0,1000,HEUNG YUEN RESTAURANT,3279 22nd St,San Francisco,CA,94110,37.755282,-122.420493,-9999


We will first rename that column to `bid` for simplicity.

In [ ]:
bus = bus.rename(columns = {'business id column': 'bid'})

Examining the entries in `bus`, is the `bid` unique for each record (i.e. each row of data)? Your code should compute the answer, i.e. don't just hard code `True` or `False`.

In [ ]:
def duplicates_check(column):
    """Take in a column of a dataframe and returns whether are duplicates"""
    if column.value_counts().sum() == len(column.unique()):
        print(f"There are no duplicates.")
        return False
    else:
        print(f"There are duplicates.")
        return True # Optional but good practice since you can use the function later in the broader code structure.
    
duplicates_check(bus['bid']);
# Adding the ; -> not display the result of the last expression.

There are no duplicates.


### Question 1b

We will now work with some important fields in `bus`.

1. Assign `top_names` to an iterable containing the top 5 most frequently used business names, from most frequent to least frequent. 
2. Assign `top_addresses` to an iterable containing the top 5 addresses where businesses are located, from most popular to least popular.
   
Recall that "an iterable object is anything that can be passed to the built-in iter function. Iterables include sequence values such as strings and tuples, and other containers such as sets and dictionaries."

**Hint 1**: You may find [value_counts](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html) helpful.

**Hint 2**: You'll need to get the names / addresses, NOT the counts associated with each. Some way to **reset the index** would come in handy. If you're unsure how to do this, try looking through the class notes or using a search engine. Part of the goal of this course is to develop independent thinking in the context of the data science lifecycle, which can involve a fair bit of exploring and reading documentation. It may be a bit annoying at first, but you'll get the hang of it, and we're here to guide you on that path! 

**Hint 3**: To check your answer, `top_names[0]` should return the string `Peet's Coffee & Tea`. It should not be a number.

In [ ]:
top_names = bus['name'] \
    .value_counts() \
    .sort_values(ascending=False) \
    .to_frame() \
    .head(5) \
    .reset_index()['name'] # or .index 
# without the last row this would be wrong since we don't want the counts just the names

top_names

0    Peet's Coffee & Tea
1       Starbucks Coffee
2             McDonald's
3            Jamba Juice
4              STARBUCKS
Name: name, dtype: object

In [ ]:
top_addresses = bus['address'] \
    .value_counts() \
    .sort_values(ascending = False) \
    .head(5) \
    .reset_index()['address']
top_addresses

0      Off The Grid
1       428 11th St
2    2948 Folsom St
3     3251 20th Ave
4           Pier 41
Name: address, dtype: object

In [ ]:
# Check:
top_names[0] # correct for both .reset_index and .index

"Peet's Coffee & Tea"

---
## Question 2. Cleaning the Business Data Postal Codes

The business data contains postal code information that we can use to aggregate the ratings over regions of the city. Let's examine and clean the postal code field. The postal code (sometimes also called a [ZIP code](https://en.wikipedia.org/wiki/ZIP_Code)) partitions the city into regions:

<img src="https://www.usmapguide.com/wp-content/uploads/2019/03/printable-san-francisco-zip-code-map.jpg" alt="ZIP Code Map" style="width: 200px">

### Question 2a. How many restaurants are in each ZIP code? 

In the cell below, create a **series** where the index is the postal code and the value is the number of records with that postal code. The series should be in descending order of count. Do you notice any odd/invalid zip codes?

In [ ]:
postal_code_series = bus['postal_code'].value_counts().sort_values(ascending = False)
postal_code_series

postal_code
94103         562
94110         555
94102         456
94107         408
94133         398
             ... 
94105-2907      1
95112           1
941             1
941102019       1
94123-3106      1
Name: count, Length: 63, dtype: int64

### Question 2b

In question 2a we noticed a large number of potentially invalid ZIP codes (e.g., "Ca").  These are likely due to data entry errors.  To get a better understanding of the potential errors in the zip codes, let's break down the problem into two parts.

#### Part I

Import a list of valid San Francisco ZIP codes by using `pd.read_json` to load the file `data/sf_zipcodes.json`, and store them as a Series in `valid_zips`. As you may expect, `pd.read_json` works similarly to `pd.read_csv` but for JSON files (a different file format you'll learn more about in HW 3) that you can read more about [here](https://pandas.pydata.org/docs/reference/api/pandas.read_json.html). If you are unsure of what data type a variable is, remember you can do `type(some_var_name)` to check!

In [ ]:
zip = pd.read_json('sf_zipcodes.json')
zip.head(1)

,zip_codes
0,94102


Observe that `pd.read_json` reads data as integers by default. This isn't quite what we want! We would like to store zip codes as strings (you'll learn more about why soon!). To do that, we can use the `astype` function to generate a copy of the Pandas `Series` stored as strings instead.

#### Part II

Construct a `DataFrame` containing only the businesses which DO NOT have valid ZIP codes.

In [ ]:
# Converting valid SF zip codes into strings using astype
valid_zips = zip['zip_codes'].astype(str)
valid_zips.head(5)

0    94102
1    94103
2    94104
3    94105
4    94107
Name: zip_codes, dtype: object

In [ ]:
# Using isin -> create a subset of data from bus w zipcodes bl in valid_zips
bus_valid_codes = bus['postal_code'].astype(str).isin(valid_zips)
# astype in 1 line -> X have to create a new var just to convert bus[postal code] to str
valid_bus = bus[bus_valid_codes]
valid_bus.head(2)

,bid,name,address,city,state,postal_code,latitude,longitude,phone_number
0,1000,HEUNG YUEN RESTAURANT,3279 22nd St,San Francisco,CA,94110,37.755282,-122.420493,-9999
1,100010,ILLY CAFFE SF_PIER 39,PIER 39 K-106-B,San Francisco,CA,94133,-9999.000000,-9999.000000,14154827284


In [ ]:
# Check if the omitted rows are actually invalid
# Use ~ 4 inverse
bus_invalid_codes = ~bus['postal_code'].astype(str).isin(valid_zips)
invalid_bus = bus[bus_invalid_codes]
invalid_bus.head(5)

,bid,name,address,city,state,postal_code,latitude,longitude,phone_number
22,100126,Lamas Peruvian Food Truck,Private Location,San Francisco,CA,-9999,-9999.0,-9999.0,-9999
68,100417,"COMPASS ONE, LLC",1 MARKET ST. FL,San Francisco,CA,94105-1420,-9999.0,-9999.0,14154324000
96,100660,TEAPENTER,1518 IRVING ST,San Francisco,CA,94122-1909,-9999.0,-9999.0,14155868318
109,100781,LE CAFE DU SOLEIL,200 FILLMORE ST,San Francisco,CA,94117-3504,-9999.0,-9999.0,14155614215
144,101084,Deli North 200,1 Warriors Way Level 300 North East,San Francisco,CA,94518,-9999.0,-9999.0,-9999


When dealing with postal codes, it's a good practice to ensure consistency in data types to avoid unexpected behavior. If one column contains string-type postal codes, it's advisable to convert the other column to the string type as well before using isin. This helps in preventing potential issues related to data type mismatches.

In the provided example, both the 'postal_code' column in the 'bus' DataFrame and the 'valid_zips' Series are explicitly converted to strings using astype(str) to ensure consistency in data types before performing the comparison.

### Question 2c

In the previous question, many of the businesses had a common invalid postal code that was likely used to encode a MISSING postal code.  Do they all share a potentially "interesting address"? For that purpose, in the following cells, we will construct a series that counts the number of businesses at each `address` that have this single likely MISSING postal code value. 

Let's break this down into steps: 

#### Part 1
Identify the single common missing postal code and assign it to `missing_postal_code`. Then create a Dataframe, `bus_missing`, to store only those businesses in `bus` that have `missing_postal_code` as their postal code.

In [ ]:
invalid_bus['postal_code'].value_counts().head(5)

postal_code
-9999    194
CA         2
94301      2
94013      2
94101      2
Name: count, dtype: int64

In [ ]:
# The missing value is clearly -9999
missing_postal_code = invalid_bus['postal_code'].value_counts().index[0]
missing_postal_code

'-9999'

In [ ]:
bus_missing = bus[bus['postal_code'] == missing_postal_code]
bus_missing.tail(2)

,bid,name,address,city,state,postal_code,latitude,longitude,phone_number
6078,98788,333 Truck,Off The Grid,San Francisco,CA,-9999,-9999.0,-9999.0,-9999
6174,99376,Mashallah Halal Food truck Ind,Off The Grid,San Francisco,CA,-9999,-9999.0,-9999.0,-9999


#### Part 2
Using `bus_missing`, find the number of businesses at each address (which would all share the same postal code). Specifically, `missing_zip_address_count` should store a Series with addresses as the indices and the counts as the values.

In [ ]:
missing_zip_address_count = (
    bus_missing.groupby('address')['postal_code']
    .value_counts()
    .sort_values(ascending=False)
    .reset_index(level='postal_code', drop=True)
)

missing_zip_address_count

address
Off The Grid                                39
Off the Grid                                10
OTG                                          4
Approved Locations                           3
Approved Private Locations                   3
                                            ..
24 Willie Mays Pl Room 2142                  1
24 Willie Mays Pl Rm 2103                    1
24 Willie Mays Pl Field Level Rm 1.11.11     1
2399 Van Ness Ave                            1
Various Farmers Markets                      1
Name: count, Length: 135, dtype: int64

You don't need the `reset_index(level='postal_code', drop=True)` line but if you wanna have the exact format specified in the question then u do

The `level` parameter in the reset_index method is used to specify which level or levels of a multi-level index should be reset. In the given context, when you have a multi-level index with levels 'address' and 'postal_code', using level='postal_code' means that you want to reset only the 'postal_code' level, leaving 'address' as part of the index. 

If you use `drop=False` (or don't specify drop), it will keep 'postal_code' as a regular column in the DataFrame. This is useful if you want to retain both levels as columns.

If you use `drop=True`, it discards the 'postal_code' level and leaves only 'address' as the index. This is useful when you want a cleaner representation, and 'postal_code' is not needed as a separate column in this context.

### Question 2e

Examine the `invalid_zip_bus` Dataframe we computed in question 2c and look at the businesses that DO NOT have the special MISSING ZIP code value. Some of the invalid postal codes are just the full 9 digit code rather than the first 5 digits. Create a new column named `postal5` in the original `bus` Dataframe which contains only the first 5 digits of the `postal_code` column.

Then, for any of the `postal5` ZIP code entries that were not a valid San Francisco ZIP Code (according to `valid_zips`), the provided code will set the `postal5` value to `None`. 

**Hint:** You will find `str` accessors particularly useful. They allow you to use your usual Python string functions in tandem with a Dataframe. For example, if you wanted to use the `replace` function on every entry in a column of a Dataframe to change the letter 'a' to 'e', you could do so by writing `df['col_name'].str.replace('a', 'e')`. Think about the different ways you can extract the first 5 digits using regular Python code!

**Do not modify the provided code! Simply add your own code in place of the ellipses.**

In [ ]:
invalid_bus.head()

,bid,name,address,city,state,postal_code,latitude,longitude,phone_number
22,100126,Lamas Peruvian Food Truck,Private Location,San Francisco,CA,-9999,-9999.0,-9999.0,-9999
68,100417,"COMPASS ONE, LLC",1 MARKET ST. FL,San Francisco,CA,94105-1420,-9999.0,-9999.0,14154324000
96,100660,TEAPENTER,1518 IRVING ST,San Francisco,CA,94122-1909,-9999.0,-9999.0,14155868318
109,100781,LE CAFE DU SOLEIL,200 FILLMORE ST,San Francisco,CA,94117-3504,-9999.0,-9999.0,14155614215
144,101084,Deli North 200,1 Warriors Way Level 300 North East,San Francisco,CA,94518,-9999.0,-9999.0,-9999


Steps:
1. initialising the 'postal5' column with None
2. then filling it with the first 5 digits of the 'postal_code' column
3. and finally, setting 'postal5' to None for entries not in the list of valid ZIP codes. 

In [ ]:
bus['postal5'] = None

# Fill in the below
bus['postal5'] = bus['postal_code'].astype(str).str[:5]
# End

bus.loc[~bus['postal5'].isin(valid_zips), 'postal5'] = None
# Checking the corrected postal5 column
bus.loc[invalid_bus.index, ['bid', 'name', 'postal_code', 'postal5']]

,bid,name,postal_code,postal5
22,100126,Lamas Peruvian Food Truck,-9999,None
68,100417,"COMPASS ONE, LLC",94105-1420,94105
96,100660,TEAPENTER,94122-1909,94122
109,100781,LE CAFE DU SOLEIL,94117-3504,94117
144,101084,Deli North 200,94518,None
...,...,...,...,...
6173,99369,HOTEL BIRON,94102-5917,94102
6174,99376,Mashallah Halal Food truck Ind,-9999,None
6199,99536,FAITH SANDWICH #2,94105-2907,94105
6204,99681,Twister,95112,None


---
## Question 3. Investigate the Inspection Data

Let's now turn to the inspection DataFrame. Earlier, we found that `ins` has 4 columns named 
`iid`, `score`, `date` and `type`.  In this section, we determine the granularity of `ins` and investigate the kinds of information provided for the inspections. 

In [ ]:
ins.head(5)

,iid,date,score,type
0,100010_20190329,03/29/2019 12:00:00 AM,-1,New Construction
1,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled
2,100017_20190417,04/17/2019 12:00:00 AM,-1,New Ownership
3,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled
4,100017_20190826,08/26/2019 12:00:00 AM,-1,Reinspection/Followup


### Question 3a
The column `iid` probably corresponds to an inspection id.  Write an expression (line of code) that evaluates to `True` or `False` based on whether all the inspection ids are unique.

**Hint:** This is a very similar question to Question 1a.

In [ ]:
duplicates_check(ins['iid']);

There are no duplicates.


### Question 3b

We would like to extract `bid` from each row of the `ins` Dataframe. If we look carefully, the column `iid` of the `ins` DataFrame appears to be the composition of two numbers and the first number looks like a business id.  

Create a new column called `bid` in the `ins` Dataframe containing just the business id.  You will want to use `ins['iid'].str` operations to do this. (Python's in-built `split` method could come in use, read up on the documentation [here](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.split.html)!) Also be sure to convert the type of this column to `int`. 

**Hint**: Similar to an earlier problem where we used `astype("string")` to convert a column to a String, here you should use `astype` to convert the `bid` column into type `int`. **No Python `for` loops or list comprehensions are allowed.** This is on the honor system since our autograder isn't smart enough to check, but if you're using `for` loops or list comprehensions, you're doing the HW incorrectly. `

In [ ]:
ins['bid'] = ins['iid'] \
    .str.split(pat='_') \
    .str[0] \
    .astype(int)
ins.head(1)

,iid,date,score,type,bid
0,100010_20190329,03/29/2019 12:00:00 AM,-1,New Construction,100010


### Question 3c

For this part, we're going to explore some new somewhat strange syntax that we haven't seen in lecture. Don't panic! If you're not sure what to do, try experimenting, Googling, and don't shy away from talking to other students or course staff.

For this problem we'll use the time component of the inspection data.  All of this information is given in the `date` column of the `ins` Dataframe. 

**No Python `for` loops or list comprehensions are allowed!**

#### Part I

What is the type of the individual `ins['date']` entries? You may want to grab the very first entry and use the `type` function in Python. 

In [ ]:
type(ins['date'].loc[0])
# string

str

#### Part II
Rather than the type you discovered in Part 1, we want each entry in `pd.TimeStamp` format. You might expect that the usual way to convert something from it current type to `TimeStamp` would be to use `astype`. You can do that, but the more typical way is to use `pd.to_datetime`. Using `pd.to_datetime`, create a new `ins['timestamp']` column containing `pd.Timestamp` objects.  These will allow us to do date manipulation with much greater ease in part III and part IV. 

Note: You may run into a UserWarning in case you do not specify the date format when using `pd.to_datetime`. To resolve this, consider using the following string '%m/%d/%Y %I:%M:%S %p' to specify the `format`.

In [ ]:
ins['timestamp'] = pd.to_datetime(ins['date'], format='%m/%d/%Y %I:%M:%S %p')
ins.head(1)

,iid,date,score,type,bid,timestamp
0,100010_20190329,03/29/2019 12:00:00 AM,-1,New Construction,100010,2019-03-29


#### Part III

What are the earliest and latest dates in our inspection data?  

**Hint**: you can use `min` and `max` on dates of the correct type.

In [ ]:
print(f"Earliest date: {ins['timestamp'].min()}")
print(f"Latest date: {ins['timestamp'].max()}")

Earliest date: 2016-10-04 00:00:00
Latest date: 2019-11-28 00:00:00


#### Part IV

We probably want to examine the inspections by year. Create an additional `ins['year']` column containing just the year of the inspection.  Consider using `pd.Series.dt.year` to do this.

In case you're curious, the documentation for `TimeStamp` data can be found at [this link](https://pandas.pydata.org/docs/reference/api/pandas.Timestamp.html#pandas.Timestamp).

In [ ]:
ins['year'] = pd.to_datetime(ins['date'], format='%m/%d/%Y %I:%M:%S %p').dt.year
ins.head()

,iid,date,score,type,bid,timestamp,year
0,100010_20190329,03/29/2019 12:00:00 AM,-1,New Construction,100010,2019-03-29,2019
1,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled,100010,2019-04-03,2019
2,100017_20190417,04/17/2019 12:00:00 AM,-1,New Ownership,100017,2019-04-17,2019
3,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled,100017,2019-08-16,2019
4,100017_20190826,08/26/2019 12:00:00 AM,-1,Reinspection/Followup,100017,2019-08-26,2019


---

## Question 4: Some Analysis

Let's try and figure out whether there are any differences between names of restaurants located in even and odd zipcodes (specifically using the 5-digit postal codes). We will break down this analysis into steps with the end goal of figuring out the restaurant with the longest name and a valid phone number, among all the even zipcodes and odd zipcodes respectively.

### Question 4a

First, create a new column `name_length` that stores the length of the `name` of each of the restaurants in `bus`. Again, **do not use for loops or list comprehensions**.

In [ ]:
bus['name_length'] = bus['name'].str.len()
bus.head(1)

,bid,name,address,city,state,postal_code,latitude,longitude,phone_number,postal5,name_length
0,1000,HEUNG YUEN RESTAURANT,3279 22nd St,San Francisco,CA,94110,37.755282,-122.420493,-9999,94110,21


### Question 4b

In order to work the 5-digit zipcodes and check whether they are even or odd, we need to ensure that there are no None values contained. Create a new Dataframe `bus_valid` which only contains rows with `postal5` values that are not None. You may find the `.isna()` function useful! For the rest of this question, we will be working with `bus_valid`.

In [ ]:
bus_valid = bus.loc[(bus['postal5'].isin(valid_zips)) & (bus['postal5'] != 'None'), :]
bus_valid.head(1)

,bid,name,address,city,state,postal_code,latitude,longitude,phone_number,postal5,name_length
0,1000,HEUNG YUEN RESTAURANT,3279 22nd St,San Francisco,CA,94110,37.755282,-122.420493,-9999,94110,21


### Question 4c

Now, assign `is_even` to a boolean **series** that indicates whether the corresponding 5-digit zipcode in `bus_valid` is even or odd. Remember to keep in mind the data type of `postal5`!

Hint: You might find the mod operation `%` useful here!

In [ ]:
# we need to convert it to numeric (integer) type before performing the modulo operation
bus_valid['postal5'] = bus_valid['postal5'].astype(int)
is_even = (bus_valid['postal5'] % 2 == 0)
is_even # type is series

0        True
1       False
2       False
3        True
4        True
        ...  
6248     True
6249    False
6250     True
6251    False
6252     True
Name: postal5, Length: 6032, dtype: bool

### Question 4d

Using the series you created above, store the name of the business with the longest name amongst all businesses located in even zipcodes in `longest_name_even`. You do not have to use the skeleton code provided and can use more/fewer lines than provided, but make sure that `longest_name_even` contains a **string** with your answer. 

In [ ]:
longest_name_even = bus_valid.loc[bus_valid['name_length'] == bus_valid['name_length'].max(), 'name'].iloc[0]
longest_name_even


'94686 Derby Grill/Giuseppe Bassuro/Bar San Pancho/Gilroy Garlic Fries'

# PART II.
After this homework, you should be comfortable with:
* Reading `pandas` documentation and using `pandas` methods,
* Working with data at different levels of granularity,
* Using `groupby` with different aggregation functions,
* Chaining different `pandas` functions and methods to find answers to exploratory questions.

___

## Question 1.

### Question 1a

In [ ]:
ins.head(5)

,iid,date,score,type,bid,timestamp,year
0,100010_20190329,03/29/2019 12:00:00 AM,-1,New Construction,100010,2019-03-29,2019
1,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled,100010,2019-04-03,2019
2,100017_20190417,04/17/2019 12:00:00 AM,-1,New Ownership,100017,2019-04-17,2019
3,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled,100017,2019-08-16,2019
4,100017_20190826,08/26/2019 12:00:00 AM,-1,Reinspection/Followup,100017,2019-08-26,2019


To better understand how the scores have been allocated, examine how the maximum score varies for each type of inspection. Create a `DataFrame` object `ins_score_by_type`, indexed by all the inspection types (e.g., New Construction, Routine - Unscheduled, etc.), with a single column named `max_score` containing the highest score received. You may find `pd.rename()` to be useful!

In [ ]:
ins_score_by_type = ins['type'].value_counts().to_frame()
ins_score_by_type = ins_score_by_type.rename(columns = {'count': 'max_score'})
ins_score_by_type.head(2)

,max_score
type,
Routine - Unscheduled,14077
Reinspection/Followup,6439


### Question 1b

In [ ]:
ins['score'].value_counts().head()

score
-1      12632
 100     1993
 96      1681
 92      1260
 94      1250
Name: count, dtype: int64

There are a large number of inspections with a `score` of `-1`. These are probably missing values. Let's see what types of inspections have scores and which do not (score of -1).  We have defined for you a new column `'Missing Score'` that shows `True` if the score for that business is `-1` to help you out with the analysis. 

Use `.groupby` to find out the number of scores that every combination of `type` and `Missing Score` can take on. The result should be a **`DataFrame`** that should look **exactly** as shown below:

<center> <img src="1b.png" width="200"/> 

In [ ]:
ins['Missing Score'] = (ins['score'] == -1).astype("str")
ins_missing_score_group = ins.groupby(['type', 'Missing Score'])['Missing Score'].value_counts().reset_index()

ins_missing_score_group.head(2)


,type,Missing Score,count
0,Administrative or Document Review,True,4
1,Community Health Assessment,True,1


### Question 1c

Using `groupby` to perform the above analysis gave us a `DataFrame` that wasn't the most readable at first glance. There are better ways to represent the above information that take advantage of the fact that we are looking at combinations of two variables. It's time to pivot (pun intended)!

Create the following `DataFrame`, and assign it to to the variable `ins_missing_score_pivot`. You'll want to use the `pivot_table` method of the `DataFrame` class, which you can read about in the pivot_table [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot_table.html). Once you create `ins_missing_score_pivot`, add another column titled `'Total'`, which contains the total number of inspections of that `type`. Sort the table by descending order of `'Total'`.

**Hint:** Consider what happens if there are no values that correspond to a particular combination of `'Missing Score'` and `'type'`. Looking at the documentation for `pivot_table`, is there any function argument that allows you to specify what value to fill in?

If you've done everything right, you should observe that inspection scores appear only to be assigned to `Routine - Unscheduled` inspections and that `ins_missing_score_pivot` looks exactly like below:


<table border="1" class="dataframe" >  <thead>    
    <tr style="text-align: right;">      <th>Missing Score</th>      <th>False</th>      <th>True</th>      <th>Total</th>    </tr>    <tr align="right">      <th>type</th>      <th></th>      <th></th>      <th></th>    </tr>  </thead>  <tbody>    
    <tr  align="right">      <th>Routine - Unscheduled</th>      <td>14031</td>      <td>46</td>      <td>14077</td>    </tr>    
    <tr  align="right">      <th>Reinspection/Followup</th>      <td>0</td>      <td>6439</td>      <td>6439</td>    </tr>    
    <tr  align="right">      <th>New Ownership</th>      <td>0</td>      <td>1592</td>      <td>1592</td>    </tr>    
    <tr  align="right">      <th>Complaint</th>      <td>0</td>      <td>1458</td>      <td>1458</td>    </tr>    
    <tr  align="right">      <th>New Construction</th>      <td>0</td>      <td>994</td>      <td>994</td>    </tr>    
    <tr  align="right">      <th>Non-inspection site visit</th>      <td>0</td>      <td>811</td>      <td>811</td>    </tr>    
    <tr  align="right">      <th>New Ownership - Followup</th>      <td>0</td>      <td>499</td>      <td>499</td>    </tr>    
    <tr  align="right">      <th>Structural Inspection</th>      <td>0</td>      <td>394</td>      <td>394</td>    </tr>    
    <tr  align="right">      <th>Complaint Reinspection/Followup</th>      <td>0</td>      <td>227</td>      <td>227</td>    </tr>    
    <tr  align="right">      <th>Foodborne Illness Investigation</th>      <td>0</td>      <td>115</td>      <td>115</td>    </tr>    
    <tr  align="right">      <th>Routine - Scheduled</th>      <td>0</td>      <td>46</td>      <td>46</td>    </tr>    
    <tr  align="right">      <th>Administrative or Document Review</th>      <td>0</td>      <td>4</td>      <td>4</td>    </tr>    
    <tr  align="right">      <th>Multi-agency Investigation</th>      <td>0</td>      <td>3</td>      <td>3</td>    </tr>    
    <tr  align="right">      <th>Special Event</th>      <td>0</td>      <td>3</td>      <td>3</td>    </tr>    
    <tr  align="right">      <th>Community Health Assessment</th>      <td>0</td>      <td>1</td>      <td>1</td>    </tr>  </tbody></table>


In [ ]:
ins_missing_score_pivot = pd.pivot_table(
    ins_missing_score_group, 
    index = 'type',
    values = 'count',
    columns = 'Missing Score',
    aggfunc = 'sum').fillna(0)

ins_missing_score_pivot['Total'] = ins_missing_score_pivot['False'] + ins_missing_score_pivot['True']
ins_missing_score_pivot.sort_values(by = 'Total', ascending = False)

Missing Score,False,True,Total
type,,,
Routine - Unscheduled,14031.0,46.0,14077.0
Reinspection/Followup,0.0,6439.0,6439.0
New Ownership,0.0,1592.0,1592.0
Complaint,0.0,1458.0,1458.0
New Construction,0.0,994.0,994.0
Non-inspection site visit,0.0,811.0,811.0
New Ownership - Followup,0.0,499.0,499.0
Structural Inspection,0.0,394.0,394.0
Complaint Reinspection/Followup,0.0,227.0,227.0


Notice that inspection scores appear only to be assigned to `Routine - Unscheduled` inspections. It is reasonable for inspection types such as `New Ownership` and `Complaint` to have no associated inspection scores, but we might be curious why there are no inspection scores for the `Reinspection/Followup` inspection type.

---

## Question 2
In this question, we will start to connect data across multiple tables. We will be using the `merge` function. 

### Question 2a

Let's figure out which restaurants had the lowest scores. Before we proceed, let's filter out missing scores from `ins` so that negative scores don't influence our results. 

Note that there might be something interesting we could learn from businesses with missing scores, but we are omitting such analysis from this homework. You might consider exploring this for the optional question at the end. 

Note: We have no idea if there is actually anything interesting to learn as we have not attempted this ourselves.

In [ ]:
ins = ins[ins["score"] > 0]
ins

,iid,date,score,type,bid,timestamp,year,Missing Score
1,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled,100010,2019-04-03,2019,False
3,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled,100017,2019-08-16,2019,False
15,100041_20190520,05/20/2019 12:00:00 AM,83,Routine - Unscheduled,100041,2019-05-20,2019,False
20,100055_20190425,04/25/2019 12:00:00 AM,98,Routine - Unscheduled,100055,2019-04-25,2019,False
21,100055_20190912,09/12/2019 12:00:00 AM,82,Routine - Unscheduled,100055,2019-09-12,2019,False
...,...,...,...,...,...,...,...,...
26654,999_20170714,07/14/2017 12:00:00 AM,77,Routine - Unscheduled,999,2017-07-14,2017,False
26656,999_20180123,01/23/2018 12:00:00 AM,80,Routine - Unscheduled,999,2018-01-23,2018,False
26660,999_20190909,09/09/2019 12:00:00 AM,80,Routine - Unscheduled,999,2019-09-09,2019,False
26661,99_20171207,12/07/2017 12:00:00 AM,82,Routine - Unscheduled,99,2017-12-07,2017,False


In [ ]:
bus

,bid,name,address,city,state,postal_code,latitude,longitude,phone_number,postal5,name_length
0,1000,HEUNG YUEN RESTAURANT,3279 22nd St,San Francisco,CA,94110,37.755282,-122.420493,-9999,94110,21
1,100010,ILLY CAFFE SF_PIER 39,PIER 39 K-106-B,San Francisco,CA,94133,-9999.000000,-9999.000000,14154827284,94133,21
2,100017,AMICI'S EAST COAST PIZZERIA,475 06th St,San Francisco,CA,94103,-9999.000000,-9999.000000,14155279839,94103,27
3,100026,LOCAL CATERING,1566 CARROLL AVE,San Francisco,CA,94124,-9999.000000,-9999.000000,14155860315,94124,14
4,100030,OUI OUI! MACARON,2200 JERROLD AVE STE C,San Francisco,CA,94124,-9999.000000,-9999.000000,14159702675,94124,16
...,...,...,...,...,...,...,...,...,...,...,...
6248,99948,SUSIECAKES BAKERY,3509 CALIFORNIA ST,San Francisco,CA,94118,-9999.000000,-9999.000000,14150452253,94118,17
6249,99988,HINODEYA SOMA,303 02nd ST STE 102,San Francisco,CA,94107,-9999.000000,-9999.000000,-9999,94107,13
6250,99991,TON TON,422 GEARY ST,San Francisco,CA,94102,-9999.000000,-9999.000000,14155531280,94102,7
6251,99992,URBAN EXPRESS KITCHENS LLC,475 06th ST,San Francisco,CA,94103,-9999.000000,-9999.000000,14150368085,94103,26


In [ ]:
# Converting business id column to all number in ins
ins['bid'] = ins['bid'].astype(int)
ins

,iid,date,score,type,bid,timestamp,year,Missing Score
1,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled,100010,2019-04-03,2019,False
3,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled,100017,2019-08-16,2019,False
15,100041_20190520,05/20/2019 12:00:00 AM,83,Routine - Unscheduled,100041,2019-05-20,2019,False
20,100055_20190425,04/25/2019 12:00:00 AM,98,Routine - Unscheduled,100055,2019-04-25,2019,False
21,100055_20190912,09/12/2019 12:00:00 AM,82,Routine - Unscheduled,100055,2019-09-12,2019,False
...,...,...,...,...,...,...,...,...
26654,999_20170714,07/14/2017 12:00:00 AM,77,Routine - Unscheduled,999,2017-07-14,2017,False
26656,999_20180123,01/23/2018 12:00:00 AM,80,Routine - Unscheduled,999,2018-01-23,2018,False
26660,999_20190909,09/09/2019 12:00:00 AM,80,Routine - Unscheduled,999,2019-09-09,2019,False
26661,99_20171207,12/07/2017 12:00:00 AM,82,Routine - Unscheduled,99,2017-12-07,2017,False


Heres the thing if i were to merge bus and ins now it wouldnt work since the bid column is not unique (aka there are multiple entries for 1 restaurant/bid 14k rows for ins, 6k something for bus)
=> MUST FIND MEDIAN SCORE

Must X remove all restaurants rows except for the median since we need to access the inv score later on.
-> fill all emdian score with the same value for a restaurant.

In [ ]:
ins_median = ins.groupby('bid')['score'].transform('median')
ins_median

1        100.0
3         91.0
15        83.0
20        90.0
21        90.0
         ...  
26654     80.0
26656     80.0
26660     80.0
26661     83.0
26662     83.0
Name: score, Length: 14031, dtype: float64

In [ ]:
ins['median_score'] = ins_median
ins

,iid,date,score,type,bid,timestamp,year,Missing Score,median_score
1,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled,100010,2019-04-03,2019,False,100.0
3,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled,100017,2019-08-16,2019,False,91.0
15,100041_20190520,05/20/2019 12:00:00 AM,83,Routine - Unscheduled,100041,2019-05-20,2019,False,83.0
20,100055_20190425,04/25/2019 12:00:00 AM,98,Routine - Unscheduled,100055,2019-04-25,2019,False,90.0
21,100055_20190912,09/12/2019 12:00:00 AM,82,Routine - Unscheduled,100055,2019-09-12,2019,False,90.0
...,...,...,...,...,...,...,...,...,...
26654,999_20170714,07/14/2017 12:00:00 AM,77,Routine - Unscheduled,999,2017-07-14,2017,False,80.0
26656,999_20180123,01/23/2018 12:00:00 AM,80,Routine - Unscheduled,999,2018-01-23,2018,False,80.0
26660,999_20190909,09/09/2019 12:00:00 AM,80,Routine - Unscheduled,999,2019-09-09,2019,False,80.0
26661,99_20171207,12/07/2017 12:00:00 AM,82,Routine - Unscheduled,99,2017-12-07,2017,False,83.0


In [ ]:
# Now merging unique-rows ins with bus
ins_named = pd.merge(
    left = ins,
    right = bus,
    how = 'inner',
    on = 'bid'
)
ins_named.head()

,iid,date,score,type,bid,timestamp,year,Missing Score,median_score,name,address,city,state,postal_code,latitude,longitude,phone_number,postal5,name_length
0,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled,100010,2019-04-03,2019,False,100.0,ILLY CAFFE SF_PIER 39,PIER 39 K-106-B,San Francisco,CA,94133,-9999.0,-9999.0,14154827284,94133,21
1,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled,100017,2019-08-16,2019,False,91.0,AMICI'S EAST COAST PIZZERIA,475 06th St,San Francisco,CA,94103,-9999.0,-9999.0,14155279839,94103,27
2,100041_20190520,05/20/2019 12:00:00 AM,83,Routine - Unscheduled,100041,2019-05-20,2019,False,83.0,UNCLE LEE CAFE,3608 BALBOA ST,San Francisco,CA,94121,-9999.0,-9999.0,-9999,94121,14
3,100055_20190425,04/25/2019 12:00:00 AM,98,Routine - Unscheduled,100055,2019-04-25,2019,False,90.0,Twirl and Dip,335 Martin Luther King Jr. Dr,San Francisco,CA,94118,-9999.0,-9999.0,14155300260,94118,13
4,100055_20190912,09/12/2019 12:00:00 AM,82,Routine - Unscheduled,100055,2019-09-12,2019,False,90.0,Twirl and Dip,335 Martin Luther King Jr. Dr,San Francisco,CA,94118,-9999.0,-9999.0,14155300260,94118,13


### Question 2b

Let's look at the 20 businesses in `ins_named` with the lowest **median** score. Order your results by the median score followed by the business name to break ties. The resulting table should look like the table below.

This one is pretty challenging! Don't forget to rename the `score` column. 

**Hint**: The `agg` function can accept a dictionary as an input. See the [agg documentation](https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.core.groupby.DataFrameGroupBy.agg.html). Additionally, when thinking about what aggregation functions to use, ask yourself what value would be in the `"name"` column for each entry across the group? Can we select just one of these values to represent the whole group?

As usual, **YOU SHOULD NOT USE LOOPS OR LIST COMPREHENSIONS**. Try and break down the problem piece by piece instead, gradually chaining together different `pandas` functions. Feel free to use more than one line!

<table border="1" class="dataframe">  <thead>    
    <tr style="text-align: right;">      <th></th>      <th>name</th>      <th>median score</th>    </tr> 
    <tr  align="right">  <th align="right">bid</th>      <th></th>      <th></th>    </tr> </thead>  <tbody>    
    <tr  align="right">      <th>84590</th>      <td>Chaat Corner</td>      <td>54.0</td>    </tr>    
    <tr  align="right">        <th>90622</th>      <td>Taqueria Lolita</td>      <td>57.0</td>    </tr>    
    <tr  align="right">         <th>94351</th>      <td>VBowls LLC</td>      <td>58.0</td>    </tr>    
    <tr  align="right">          <th>69282</th>      <td>New Jumbo Seafood Restaurant</td>      <td>60.5</td>    </tr>    
    <tr  align="right">         <th>1154</th>      <td>SUNFLOWER RESTAURANT</td>      <td>63.5</td>    </tr>  
    <tr  align="right">          <th>93150</th>      <td>Chez Beesen</td>      <td>64.0</td>    </tr>   
    <tr  align="right">     <th>39776</th>      <td>Duc Loi Supermarket</td>      <td>64.0</td>    </tr>  
    <tr  align="right">         <th>78328</th>      <td>Golden Wok</td>      <td>64.0</td>    </tr>  
    <tr  align="right">          <th>69397</th>      <td>Minna SF Group LLC</td>      <td>64.0</td>    </tr>     
    <tr  align="right">        <th>93502</th>      <td>Smoky Man</td>      <td>64.0</td>    </tr>    
    <tr  align="right">           <th>98995</th>      <td>Vallarta's Taco Bar</td>      <td>64.0</td>    </tr>    
    <tr  align="right">         <th>10877</th>      <td>CHINA FIRST INC.</td>      <td>64.5</td>    </tr>    
    <tr  align="right">        <th>71310</th>      <td>Golden King Vietnamese Restaurant</td>      <td>64.5</td>    </tr>     
    <tr  align="right">          <th>89070</th>      <td>Lafayette Coffee Shop</td>      <td>64.5</td>    </tr>
    <tr  align="right">          <th>71008</th>      <td>House of Pancakes</td>      <td>65.0</td>    </tr> 
    <tr  align="right">         <th>2542</th>      <td>PETER D'S RESTAURANT</td>      <td>65.0</td>    </tr>           
    <tr  align="right">        <th>3862</th>      <td>IMPERIAL GARDEN SEAFOOD RESTAURANT</td>      <td>66.0</td>    </tr>   
    <tr  align="right">         <th>61427</th>      <td>Nick's Foods</td>      <td>66.0</td>    </tr>    
    <tr  align="right">          <th>72176</th>      <td>Wolfes Lunch</td>      <td>66.0</td>    </tr>    
    <tr  align="right">        <th>89141</th>      <td>Cha Cha Cha on Mission</td>      <td>66.5</td>    </tr>  </tbody></table>

In [ ]:
table2b = ins_named.sort_values(by=['median_score', 'name']).head(20)[['bid', 'median_score', 'name']]
table2b.head(20)

,bid,median_score,name
10285,84590,54.0,Chaat Corner
12028,90622,57.0,Taqueria Lolita
12943,94351,58.0,VBowls LLC
6953,69282,60.5,New Jumbo Seafood Restaurant
6954,69282,60.5,New Jumbo Seafood Restaurant
290,1154,63.5,SUNFLOWER RESTAURANT
291,1154,63.5,SUNFLOWER RESTAURANT
12707,93150,64.0,Chez Beesen
3423,39776,64.0,Duc Loi Supermarket
3424,39776,64.0,Duc Loi Supermarket


Let's figure out which restaurant had the worst score ever (single lowest score). 

In the cell below, assign `worst_restaurant` to the name of the restaurant with the **lowest inspection score ever**. We should not be considering restaurants with missing scores, so this should not be a retaurant that has a score of `-1`. For fun: Look up the reviews for this restaurant on Yelp. Do you see any reviews that indicate this restaurant had health inspection issues?

In [ ]:
worst_restaurant_row = ins_named.loc[(ins_named['Missing Score'] != False) & (ins_named['score'] == ins_named['score'].min()), :].reset_index()
worst_restaurant = worst_restaurant_row.loc[0, 'name']
worst_restaurant

'Lollipot'

In [ ]:
worst_restaurant_row

,index,iid,date,score,type,bid,timestamp,year,Missing Score,median_score,name,address,city,state,postal_code,latitude,longitude,phone_number,postal5,name_length
0,10898,86718_20180522,05/22/2018 12:00:00 AM,45,Routine - Unscheduled,86718,2018-05-22,2018,False,90.0,Lollipot,890 Taraval St,San Francisco,CA,94116,-9999.0,-9999.0,14155818962,94116,8


#### Question 2d

Did this restaurant clean up its act? Look in the database to see if it scored better on its next inspection. Assign `cleaned_up` to `True` or `False`, depending on whether it performed better or not.

In [ ]:
worst_restaurant_all_ratings = ins_named.loc[ins_named['name'] == worst_restaurant, :]
worst_restaurant_all_ratings

,iid,date,score,type,bid,timestamp,year,Missing Score,median_score,name,address,city,state,postal_code,latitude,longitude,phone_number,postal5,name_length
10897,86718_20161116,11/16/2016 12:00:00 AM,90,Routine - Unscheduled,86718,2016-11-16,2016,False,90.0,Lollipot,890 Taraval St,San Francisco,CA,94116,-9999.0,-9999.0,14155818962,94116,8
10898,86718_20180522,05/22/2018 12:00:00 AM,45,Routine - Unscheduled,86718,2018-05-22,2018,False,90.0,Lollipot,890 Taraval St,San Francisco,CA,94116,-9999.0,-9999.0,14155818962,94116,8
10899,86718_20181005,10/05/2018 12:00:00 AM,90,Routine - Unscheduled,86718,2018-10-05,2018,False,90.0,Lollipot,890 Taraval St,San Francisco,CA,94116,-9999.0,-9999.0,14155818962,94116,8


In [ ]:
worst_restaurant_all_ratings['cleaned_up'] = worst_restaurant_all_ratings['score'] > worst_restaurant_all_ratings['score'].shift(1)
worst_restaurant_all_ratings['cleaned_up'].fillna(False, inplace=True)
worst_restaurant_all_ratings

,iid,date,score,type,bid,timestamp,year,Missing Score,median_score,name,address,city,state,postal_code,latitude,longitude,phone_number,postal5,name_length,cleaned_up
10897,86718_20161116,11/16/2016 12:00:00 AM,90,Routine - Unscheduled,86718,2016-11-16,2016,False,90.0,Lollipot,890 Taraval St,San Francisco,CA,94116,-9999.0,-9999.0,14155818962,94116,8,False
10898,86718_20180522,05/22/2018 12:00:00 AM,45,Routine - Unscheduled,86718,2018-05-22,2018,False,90.0,Lollipot,890 Taraval St,San Francisco,CA,94116,-9999.0,-9999.0,14155818962,94116,8,False
10899,86718_20181005,10/05/2018 12:00:00 AM,90,Routine - Unscheduled,86718,2018-10-05,2018,False,90.0,Lollipot,890 Taraval St,San Francisco,CA,94116,-9999.0,-9999.0,14155818962,94116,8,True


---

## Question 3. Ice cream

---

## Question 4. Ratings over time